In [ ]:
import torch
torch.__version__

'2.1.0+cu121'

In [ ]:
!python -m pip install torch==2.1.1+cpu --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.9/184.9 MB 8.9 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu121
    Uninstalling torch-2.1.0+cu121:
      Successfully uninstalled torch-2.1.0+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 2.1.1+cpu which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.1.1+cpu which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.1.1+cpu which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 2.1.1+cpu which is incompatible.


In [ ]:
!python -m pip install intel-extension-for-pytorch==2.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 MB 32.3 MB/s eta 0:00:00


In [ ]:
!python -c "import torch; import intel_extension_for_pytorch as ipex; print(torch.__version__); print(ipex.__version__);"

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
2.1.1+cpu
2.1.0+cpu


In [ ]:
import kagglehub

kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
!pip install immutabledict sentencepiece

In [ ]:
# Choose variant and machine type
VARIANT = '7b-it-quant' #@param ['2b', '2b-it', '7b', '7b-it', '7b-quant', '7b-it-quant']
MACHINE_TYPE = 'cpu' #@param ['cuda', 'cpu']

In [ ]:
import os

# Load model weights
weights_dir = kagglehub.model_download(f'google/gemma/pyTorch/{VARIANT}')

# Ensure that the tokenizer is present
tokenizer_path = os.path.join(weights_dir, 'tokenizer.model')
assert os.path.isfile(tokenizer_path), 'Tokenizer not found!'

# Ensure that the checkpoint is present
ckpt_path = os.path.join(weights_dir, f'gemma-{VARIANT}.ckpt')
assert os.path.isfile(ckpt_path), 'PyTorch checkpoint not found!'

100%|██████████| 6.79G/6.79G [05:43<00:00, 21.2MB/s]
Extracting model files...


In [ ]:
!cp -r /root/.cache/kagglehub /content

Start here without redownloading the Gemma model:

In [ ]:
!cp -r /content/kagglehub /root/.cache

In [ ]:
import os

weights_dir = '/root/.cache/kagglehub/models/google/gemma/pyTorch/7b-it/2'
# Ensure that the tokenizer is present
tokenizer_path = os.path.join(weights_dir, 'tokenizer.model')
assert os.path.isfile(tokenizer_path), 'Tokenizer not found!'

# Ensure that the checkpoint is present
ckpt_path = os.path.join(weights_dir, f'gemma-7b-it.ckpt')
assert os.path.isfile(ckpt_path), 'PyTorch checkpoint not found!'

In [ ]:
# NOTE: The "installation" is just cloning the repo.
!git clone https://github.com/google/gemma_pytorch.git

fatal: destination path 'gemma_pytorch' already exists and is not an empty directory.


In [ ]:
import sys

sys.path.append('gemma_pytorch')

In [ ]:
from gemma_pytorch.gemma.config import get_config_for_7b, get_config_for_2b
from gemma_pytorch.gemma.model import GemmaForCausalLM

In [ ]:
import torch

# Set up model config.
model_config = get_config_for_2b() if "2b" in VARIANT else get_config_for_7b()
model_config.tokenizer = tokenizer_path
model_config.quant = 'quant' in VARIANT

# Instantiate the model and load the weights.
torch.set_default_dtype(model_config.get_dtype())
device = torch.device(MACHINE_TYPE)
model = GemmaForCausalLM(model_config)
model.load_weights(ckpt_path)
model = model.to(device).eval()

In [ ]:
import intel_extension_for_pytorch as ipex

model = ipex.optimize(model, weights_prepack=False)

model = torch.compile(model, backend="ipex")

In [ ]:
# Generate with one request in chat mode

# Chat templates
USER_CHAT_TEMPLATE = '<start_of_turn>user\n{prompt}<end_of_turn>\n'
MODEL_CHAT_TEMPLATE = '<start_of_turn>model\n{prompt}<end_of_turn>\n'

# Sample formatted prompt
prompt = (
    USER_CHAT_TEMPLATE.format(
        prompt='coba buat jokes bapack-bapack yang garing'
    )
    # + MODEL_CHAT_TEMPLATE.format(prompt='California.')
    # + USER_CHAT_TEMPLATE.format(prompt='What can I do in California?')
    + '<start_of_turn>model\n'
)
print('Chat prompt:\n', prompt)

with torch.no_grad(), torch.inference_mode(), torch.cpu.amp.autocast(enabled=True):
  print(model.generate(
      USER_CHAT_TEMPLATE.format(prompt=prompt),
      device=device,
      output_len=100,
  ))

Chat prompt:
 <start_of_turn>user
coba buat jokes bapack-bapack yang garing<end_of_turn>
<start_of_turn>model



KeyboardInterrupt: 